<a href="https://colab.research.google.com/github/SunbirdAI/salt/blob/main/notebooks/ASR_correction_model_usage_(FLAN_T5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers
import torch

In [2]:
correction_tokenizer = transformers.T5Tokenizer.from_pretrained(
    'jq/asr-correction-flan-t5')
correction_model = transformers.T5ForConditionalGeneration.from_pretrained(
    'jq/asr-correction-flan-t5')


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [3]:
def correct(asr_text):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  inputs = correction_tokenizer(asr_text, return_tensors="pt").to(device)
  translated_tokens = correction_model.to(device).generate(
      **inputs,
      max_new_tokens=250,
      num_beams=20,
  )

  result = correction_tokenizer.batch_decode(
      translated_tokens, skip_special_tokens=True)[0]
  return result

Example of correcting raw output from ASR model.

In [5]:
correct("optionban ani n'ekyalo gyennaly eyo waliwo domestic violence ey'abafumbo")

"Option B. Ani n'ekyalo gyennaly eyo waliwo domestic violence ey'abafumbo."

Example of correcting ASR output with phonetic uncertainty (uncertain characters in brackets).

In [4]:
correct("optionban ani n'ekyalo gy(')e(n )nal(yi)(o) eyo waliwo domestic violence ey'abafumbo")

"Option B. Ani n'ekyalo gy'e, nali eyo waliwo domestic violence ey'abafumbo."

This model doesn't need the language to be specified, and handles all the SALT Ugandan languages.

In [6]:
correct("hy jus coming the hospital everything was goodnothing bad but the problem is i didn't get even single medicine that is my problem but like a toilet everything is very clean")

"He just came the hospital. Everything was good, nothing bad. But the problem is, I didn't get even one medicine that is my problem. But like a toilet, everything is very clean."